In [1]:
import json
import random
from sklearn.model_selection import train_test_split

In [2]:
data_train = json.load(open('../Dataset/NER_TRAIN_JUDGEMENT.json'))
data_test = json.load(open('../Dataset/NER_TEST_JUDGEMENT.json'))

In [3]:
def get_token_indices(sentence, start, end):
    words = sentence.split(' ')
    start_index = 0
    end_index = 0
    for i, word in enumerate(words):
        if start_index == 0 and start < len(word):
            start_index = i
        start -= len(word) + 1
        if end == len(word) or end == len(word) - 1:
            end_index = i+1
            break
        if end < len(word):
            end_index = i
            break
        end -= len(word) + 1
    return start_index, end_index

In [4]:
data_train_preprocessed = {}

for doc in data_train:
    data_train_preprocessed[doc['id']] = {}
    data_train_preprocessed[doc['id']]['text'] = doc['data']['text']
    data_train_preprocessed[doc['id']]['labels'] = ['O'] * len(doc['data']['text'].split(' '))
    
    for results in doc['annotations'][0]['result']:
        entity = results['value']
        start, end = get_token_indices(doc['data']['text'], entity['start'], entity['end'])
        if start == end:
            data_train_preprocessed[doc['id']]['labels'][start] = 'B_' + results['value']['labels'][0]
        else:
            data_train_preprocessed[doc['id']]['labels'][start] = 'B_' + results['value']['labels'][0]
            for i in range(start+1, end):
                data_train_preprocessed[doc['id']]['labels'][i] = 'I_' + results['value']['labels'][0]


In [5]:
data_test_preprocessed = {}

for doc in data_test:
    data_test_preprocessed[doc['id']] = {}
    data_test_preprocessed[doc['id']]['text'] = doc['data']['text']
    data_test_preprocessed[doc['id']]['labels'] = ['O'] * len(doc['data']['text'].split(' '))
    
    for results in doc['annotations'][0]['result']:
        entity = results['value']
        start, end = get_token_indices(doc['data']['text'], entity['start'], entity['end'])
        if start == end:
            data_test_preprocessed[doc['id']]['labels'][start] = 'B_' + results['value']['labels'][0]
        else:
            data_test_preprocessed[doc['id']]['labels'][start] = 'B_' + results['value']['labels'][0]
            for i in range(start+1, end):
                data_test_preprocessed[doc['id']]['labels'][i] = 'I_' + results['value']['labels'][0]

In [7]:
# split the data into train and validation sets with a 85:15 ratio and randomly stratified (stratification based on the labels)
# labels for each data point is a list of tags for different tokens in the sentence. There are 27 different tags in total and each data point can have multiple tags
# perform split to ensure stratification based on the tags

data_train_ids = list(data_train_preprocessed.keys())
data_train_labels = [data_train_preprocessed[id]['labels'] for id in data_train_ids]

data_train_ids, data_val_ids, data_train_labels, data_val_labels = train_test_split(data_train_ids, data_train_labels, test_size=0.15, stratify=data_train_labels, random_state=42)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9435,) + inhomogeneous part.